In [1]:
import os
import numpy as np
import pandas as pd
import pysam

In [16]:
! ls -l ../../carmelab/backup/genomes/from_Arielle/ust/Ust_Ishim.hg19_1000g.all.bam

-rw-r--r-- 1 tomerozen lirancarmel_lab 137825671438 Oct  3 17:25 ../../carmelab/backup/genomes/from_Arielle/ust/Ust_Ishim.hg19_1000g.all.bam


In [2]:
samfile = pysam.AlignmentFile("../../carmelab/backup/genomes/from_Arielle/altai/altai_bams/Altai_chr21.bam", "rb")

In [26]:
count = 0
count_not_eq = 0
for read in samfile.fetch('21', 9999999, 10000000):
    for index, base in enumerate(read.get_forward_sequence()):
        if index != read.get_aligned_pairs()[index][0]:
            count_not_eq+= 1
        count+=1

print(f"count = {count}, count_not_eq = {count_not_eq}")


count = 19441, count_not_eq = 55


In [37]:
df = pd.DataFrame(columns=['read_name', 'position_in_reference', 'position_in_read', 'after'])
# samfile.count('21')
for read in samfile.fetch('21', 9999998, 10000000):
    if read.mapping_quality < 20 and read.query_length == read.reference_length:
#     print(read.query_name)
        print("Read Start", read.query_name)
        for index, base in enumerate(read.get_forward_sequence()):
    #         print(f"{base}, {read.get_aligned_pairs()[index]}, {read.get_forward_sequence()[index]}, {'!!!!!' if base != read.get_forward_sequence()[index] else ''}")
            if read.get_reference_sequence()[index].lower() == "c" and read.query_alignment_qualities[index] < 53 and index == read.get_aligned_pairs()[index][0]:
                new_row = pd.DataFrame([{
                    "read_name":read.query_name,
                    "position_in_reference": read.get_aligned_pairs()[index][0],
                    "position_in_read": read.get_aligned_pairs()[index][1],
                    "after": base}])
                df = pd.concat([df, new_row], axis=0, ignore_index=True)
#             letter_before = 
#             df["letter_before_reference"] =
#             df["letter_before_read"] =
#             df["letter_after_reference"] = 
#             df["letter_after_read"] = 
        


Read Start SN928_0068_BB022WACXX:7:2207:19474:143107
Read Start NIOBE_0139_A_D0B5GACXX:8:1101:12862:32512
Read Start SN928_0068_BB022WACXX:3:2107:5210:28211
Read Start SN7001204_0130_AC0M6HACXX_PEdi_SS_L9302_L9303_1:7:2305:5435:70325
Read Start NIOBE_0139_A_D0B5GACXX:8:2103:9066:112404
Read Start SN928_0073_BD0J78ACXX:3:2306:4202:145371
Read Start SN7001204_0130_AC0M6HACXX_PEdi_SS_L9302_L9303_1:3:2208:18824:54223
Read Start SN928_0068_BB022WACXX:1:1302:6621:144450
Read Start SN928_0068_BB022WACXX:8:2204:16722:157956
Read Start SN928_0068_BB022WACXX:8:1207:14404:198425
Read Start SN7001204_0130_AC0M6HACXX_PEdi_SS_L9302_L9303_1:2:2104:12718:14304
Read Start SN7001204_0130_AC0M6HACXX_PEdi_SS_L9302_L9303_1:8:1102:7369:92113
Read Start SN7001204_0131_BC0M3YACXX_PEdi_SS_L9302_L9303_2:6:2211:4671:37711
Read Start SN7001204_0130_AC0M6HACXX_PEdi_SS_L9302_L9303_1:1:2215:15809:65866
Read Start SN7001204_0131_BC0M3YACXX_PEdi_SS_L9302_L9303_2:2:2112:11385:73209
Read Start SN928_0068_BB022WACXX:1:21

/usr/local/hurcs/notebook/6.4.11/lib/python3.7/site-packages/ipykernel_launcher.py:15: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  from ipykernel import kernelapp as app


Read Start SN928_0068_BB022WACXX:8:2307:17176:119782
Read Start SN928_0073_BD0J78ACXX:8:1203:16596:155766
Read Start SN928_0068_BB022WACXX:2:2101:15032:189756
Read Start NIOBE_0139_A_D0B5GACXX:8:1108:6168:108480
Read Start SN7001204_0130_AC0M6HACXX_PEdi_SS_L9302_L9303_1:5:2209:13253:13611
Read Start SN7001204_0130_AC0M6HACXX_PEdi_SS_L9302_L9303_1:3:2113:3705:42439
Read Start SN7001204_0130_AC0M6HACXX_PEdi_SS_L9302_L9303_1:7:1114:16398:15200
Read Start SN7001204_0130_AC0M6HACXX_PEdi_SS_L9302_L9303_1:2:1107:9217:41444
Read Start SN7001204_0130_AC0M6HACXX_PEdi_SS_L9302_L9303_1:3:1111:4010:50564
Read Start SN928_0073_BD0J78ACXX:6:2306:3434:197087
Read Start SN7001204_0130_AC0M6HACXX_PEdi_SS_L9302_L9303_1:1:1216:17832:49254
Read Start SN928_0068_BB022WACXX:7:1106:6384:10083
Read Start NIOBE_0139_A_D0B5GACXX:8:2201:21039:177764
Read Start SN7001204_0130_AC0M6HACXX_PEdi_SS_L9302_L9303_1:6:1311:14479:51082
Read Start NIOBE_0139_A_D0B5GACXX:8:2208:3546:12515
Read Start SN928_0073_BD0J78ACXX:2:2

Read Start SN928_0068_BB022WACXX:3:1208:14574:71897
Read Start SN928_0068_BB022WACXX:4:1108:16739:53327
Read Start SN7001204_0130_AC0M6HACXX_PEdi_SS_L9302_L9303_1:7:1205:17300:11540
Read Start SN928_0068_BB022WACXX:4:2201:19483:61848
Read Start SN7001204_0130_AC0M6HACXX_PEdi_SS_L9302_L9303_1:3:2112:13974:4471
Read Start SN7001204_0131_BC0M3YACXX_PEdi_SS_L9302_L9303_2:1:2215:3588:30575
Read Start SN7001204_0131_BC0M3YACXX_PEdi_SS_L9302_L9303_2:8:2205:15446:96927
Read Start SN7001204_0130_AC0M6HACXX_PEdi_SS_L9302_L9303_1:6:1211:8579:32493
Read Start NIOBE_0139_A_D0B5GACXX:7:1301:1736:133203
Read Start SN7001204_0130_AC0M6HACXX_PEdi_SS_L9302_L9303_1:1:2309:4651:9149
Read Start SN7001204_0131_BC0M3YACXX_PEdi_SS_L9302_L9303_2:7:1310:14697:75994
Read Start SN7001204_0131_BC0M3YACXX_PEdi_SS_L9302_L9303_2:2:2211:18518:15495
Read Start SN7001204_0130_AC0M6HACXX_PEdi_SS_L9302_L9303_1:2:2201:12968:96501
Read Start SN928_0068_BB022WACXX:6:2203:7243:163824
Read Start SN7001204_0130_AC0M6HACXX_PEdi_

In [39]:
df.shape

(712, 4)